In [ ]:
pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2


In [ ]:
pip uninstall cornac -y

Found existing installation: cornac 1.14.2
Uninstalling cornac-1.14.2:
  Successfully uninstalled cornac-1.14.2


In [ ]:
pip uninstall tensorflow==2.9.1 -y

Found existing installation: tensorflow 1.15.2
Uninstalling tensorflow-1.15.2:
  Successfully uninstalled tensorflow-1.15.2


In [ ]:
pip install scikit-learn==0.22.1 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.0 MB 9.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.22.1 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.22.1 which is incompatible.


In [ ]:
pip install cornac==1.6.1 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.0 MB 6.6 MB/s 


In [ ]:
pip install tensorflow==1.15.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110.5 MB 34 kB/s 
     |████████████████████████████████| 3.8 MB 5.9 MB/s 
     |████████████████████████████████| 503 kB 12.0 MB/s 
     |████████████████████████████████| 50 kB 7.1 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=60765a307d2d821244f1d59fdf02ee632e936d3c1a9d8384dd243d3599e72d6b
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uni

In [ ]:
# pip install --upgrade tensorflow-gpu==1.15.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410.9 MB 33 kB/s 


In [ ]:
pip install --upgrade tensorflow-gpu==1.15.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%tensorflow_version 1.15

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cornac
import cornac.eval_methods
from cornac.eval_methods import RatioSplit, base_method
from cornac.hyperopt import Continuous, Discrete, RandomSearch
from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
# 추천할 top k개의 장소
TOP_K = 10

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/score_board.csv', index_col =0)

In [ ]:
data_df['rating'] =data_df['rating'].astype('float')
data_df = data_df.rename({'u_id':'userID','p_id':'itemID'},axis=1)
#data_df = data_df.groupby('userID').filter(lambda x : len(x)>= 10).copy()
data_df = data_df.fillna(5)

In [ ]:
data_df

,itemID,rating
u_id,,
0,542,3.0
0,287,3.0
0,163,0.0
0,775,3.0
0,310,0.0
...,...,...
1727,574,5.0
1728,161,5.0
1728,553,5.0


In [ ]:
user_list=list(data_df['u_id'].unique())

In [ ]:
import random
random.seed(123)
user_list_rand30 = random.sample(user_list,30)
user_list_rand30

[107,
 548,
 178,
 1574,
 834,
 545,
 220,
 1717,
 78,
 776,
 1098,
 1151,
 680,
 697,
 106,
 326,
 276,
 690,
 1148,
 683,
 1437,
 502,
 335,
 3,
 893,
 1584,
 179,
 1223,
 773,
 143]

In [ ]:
data_df.reset_index(inplace=True)
data_df

,u_id,itemID,rating
0,0,542,3.0
1,0,287,3.0
2,0,163,0.0
3,0,775,3.0
4,0,310,0.0
...,...,...,...
5137,1727,574,5.0
5138,1728,161,5.0
5139,1728,553,5.0
5140,1729,505,5.0


In [ ]:
data = [tuple(x) for x in data_df.values]
data

[(0.0, 542.0, 3.0),
 (0.0, 287.0, 3.0),
 (0.0, 163.0, 0.0),
 (0.0, 775.0, 3.0),
 (0.0, 310.0, 0.0),
 (0.0, 739.0, 5.0),
 (0.0, 161.0, 3.0),
 (0.0, 296.0, 3.0),
 (0.0, 452.0, 3.0),
 (0.0, 121.0, 3.0),
 (0.0, 240.0, 0.0),
 (0.0, 608.0, 0.0),
 (0.0, 95.0, 3.0),
 (0.0, 162.0, 0.0),
 (0.0, 609.0, 5.0),
 (0.0, 485.0, 0.0),
 (0.0, 409.0, 5.0),
 (0.0, 348.0, 3.0),
 (0.0, 279.0, 5.0),
 (0.0, 78.0, 3.0),
 (0.0, 245.0, 3.0),
 (0.0, 162.0, 0.0),
 (0.0, 412.0, 3.0),
 (0.0, 341.0, 5.0),
 (0.0, 203.0, 5.0),
 (0.0, 341.0, 5.0),
 (0.0, 728.0, 3.0),
 (1.0, 512.0, 5.0),
 (1.0, 512.0, 5.0),
 (2.0, 633.0, 3.0),
 (3.0, 228.0, 0.0),
 (3.0, 298.0, 3.0),
 (4.0, 700.0, 3.0),
 (4.0, 498.0, 0.0),
 (5.0, 726.0, 0.0),
 (5.0, 465.0, 5.0),
 (5.0, 755.0, 3.0),
 (5.0, 146.0, 3.0),
 (5.0, 538.0, 3.0),
 (6.0, 443.0, 0.0),
 (7.0, 116.0, 5.0),
 (7.0, 294.0, 3.0),
 (7.0, 139.0, 0.0),
 (8.0, 69.0, 5.0),
 (9.0, 844.0, 5.0),
 (9.0, 824.0, 3.0),
 (9.0, 824.0, 3.0),
 (10.0, 610.0, 5.0),
 (11.0, 715.0, 5.0),
 (12.0, 15.0, 0.0),
 

In [ ]:
# Instantiate evaluation metrics
ndcg = cornac.metrics.NDCG(k=TOP_K)
pre = cornac.metrics.Precision(k=TOP_K)
rec = cornac.metrics.Recall(k=TOP_K)
fm = cornac.metrics.FMeasure(k=TOP_K)

In [ ]:

#Define an evaluation method to split feedback into train and test sets
ratio_split = RatioSplit(
    data=data,
    test_size=0.1,
    val_size=0.1,
    rating_threshold=1.0,
    seed=123,
    verbose=True,
)

neumf = cornac.models.NeuMF(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_neg=50,
    seed=123,
    early_stopping = {'min_delta':0.0, 'patience':5}
)

# RandomSearch
rs_neumf = RandomSearch(
    model = neumf,
    space=[
        Discrete("num_epochs", [50, 100, 150, 200]),
        Discrete("num_factors", [4, 8]),
        Discrete("batch_size", [128, 256, 512]),
        Continuous("lr", 0.001, 0.01)
    ],
    metric = fm,
    eval_method = ratio_split
)



rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 1535
Number of items = 780
Number of ratings = 3763
Max rating = 5.0
Min rating = 0.0
Global mean = 4.0
---
Test data:
Number of users = 252
Number of items = 204
Number of ratings = 377
Number of unknown users = 0
Number of unknown items = 0
---
Validation data:
Number of users = 240
Number of items = 201
Number of ratings = 369
---
Total users = 1535
Total items = 780


In [ ]:
# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[rs_neumf],
    metrics=[ndcg, pre, rec, fm],
).run()


[RandomSearch_NeuMF] Training started!
Evaluating: {'batch_size': 512, 'lr': 0.007416597884709045, 'num_epochs': 150, 'num_factors': 4}


  0%|          | 0/150 [00:00<?, ?it/s]

Early stopping:
- best epoch = 8, stopped epoch = 13
- best monitored value = 0.178814 (delta = -0.010890)
Evaluating: {'batch_size': 512, 'lr': 0.005961832921746022, 'num_epochs': 200, 'num_factors': 4}


  0%|          | 0/200 [00:00<?, ?it/s]

Early stopping:
- best epoch = 8, stopped epoch = 13
- best monitored value = 0.182191 (delta = -0.010087)
Evaluating: {'batch_size': 256, 'lr': 0.00982687778546154, 'num_epochs': 50, 'num_factors': 8}


...

  0%|          | 0/100 [00:00<?, ?it/s]

Early stopping:
- best epoch = 9, stopped epoch = 14
- best monitored value = 0.186714 (delta = -0.000116)
Best parameter settings: {'batch_size': 512, 'lr': 0.0057864482838717955, 'num_epochs': 50, 'num_factors': 8}
F1@10 = 0.0428

[RandomSearch_NeuMF] Evaluation started!


Ranking:   0%|          | 0/252 [00:00<?, ?it/s]

Ranking:   0%|          | 0/240 [00:00<?, ?it/s]


VALIDATION:
...
                   |  F1@10 | NDCG@10 | Precision@10 | Recall@10 | Time (s)
------------------ + ------ + ------- + ------------ + --------- + --------
RandomSearch_NeuMF | 0.0428 |  0.0942 |       0.0249 |    0.1909 |   0.4358

TEST:
...
                   |  F1@10 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
------------------ + ------ + ------- + ------------ + --------- + --------- + --------
RandomSearch_NeuMF | 0.0417 |  0.0885 |       0.0243 |    0.1956 |  513.6636 |   0.4717



In [ ]:
print('Random search: ', rs_neumf.best_params)

Random search:  {'batch_size': 512, 'lr': 0.0057864482838717955, 'num_epochs': 50, 'num_factors': 8}


In [ ]:
ratio_split = RatioSplit(
    data=data,
    test_size=0,
    rating_threshold=1.0,
    verbose=True,
)

neumf = cornac.models.NeuMF(
    num_factors = rs_neumf.best_params['num_factors'],
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs = rs_neumf.best_params['num_epochs'],
    lr=rs_neumf.best_params['lr'],
    num_neg=50,
    batch_size = rs_neumf.best_params['batch_size'],
    seed=123,
    early_stopping = {'min_delta':0.0, 'patience':5}
)

cornac.Experiment(
    eval_method=ratio_split,
    models=[neumf],
    metrics=[ndcg, pre, rec, fm],
).run()

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 1730
Number of items = 844
Number of ratings = 4604
Max rating = 5.0
Min rating = 0.0
Global mean = 4.0
---
Test data:
Number of users = 1730
Number of items = 844
Number of ratings = 4604
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 1730
Total items = 844

[NeuMF] Training started!


  0%|          | 0/50 [00:00<?, ?it/s]


[NeuMF] Evaluation started!


Ranking:   0%|          | 0/1730 [00:00<?, ?it/s]


TEST:
...
      |  F1@10 | NDCG@10 | Precision@10 | Recall@10 | Train (s) | Test (s)
----- + ------ + ------- + ------------ + --------- + --------- + --------
NeuMF | 0.2335 |  0.7053 |       0.1578 |    0.8551 |  204.7705 |   2.6812



In [ ]:
# import pandas as pd
# pred_user = pd.read_csv('../../common/data/survey_result.csv')
# pred_user = set(pred_user['u_id'])

In [ ]:
user_list_rand30

[107,
 548,
 178,
 1574,
 834,
 545,
 220,
 1717,
 78,
 776,
 1098,
 1151,
 680,
 697,
 106,
 326,
 276,
 690,
 1148,
 683,
 1437,
 502,
 335,
 3,
 893,
 1584,
 179,
 1223,
 773,
 143]

In [ ]:
all_prediction = pd.DataFrame(columns=['u_id','p_id','ncf_score'])

for u_idx, u_id in tqdm(enumerate(user_list_rand30)):
  
      tmp = pd.DataFrame(neumf.rank(u_idx)).T
      tmp['u_id'] = u_id
      tmp = pd.DataFrame(tmp,columns=['u_id',0,1])
      tmp.rename(columns={0:'p_id',1:'ncf_score'}, inplace=True)
      all_prediction = all_prediction.append(tmp)

30it [00:00, 38.44it/s]


In [ ]:
all_prediction

,u_id,p_id,ncf_score
0,107,36.0,2.492180e-01
1,107,79.0,9.585652e-02
2,107,120.0,2.111465e-03
3,107,81.0,4.743088e-01
4,107,350.0,2.498185e-01
...,...,...,...
839,143,786.0,1.370907e-06
840,143,650.0,7.688999e-06
841,143,597.0,6.324053e-04
842,143,685.0,3.576279e-07


In [ ]:
len(all_prediction['u_id']==107)

25320

In [ ]:
all_prediction.to_csv('/content/drive/MyDrive/Colab Notebooks/data/prediction/ncf_scores.csv',index=False)